In [ ]:
# use debug function for pretty printing
from devtools import debug

# configure logging
import logging

logging.basicConfig(
    format="{asctime} - {name} - {levelname} - {message}", style="{", level="INFO"
)

# show every web request
logging.getLogger("firebolt.http_client").setLevel("DEBUG")

Set an engine_name and database_name you want to operate on before proceeding

In [ ]:
engine_name = ""
database_name = ""
assert engine_name
assert database_name

### Inspect Configuration
Run the following cell to make sure you can connect to Firebolt, and that your environment variables or .env file is being read properly.

In [ ]:
from firebolt.firebolt_client import FireboltClient

with FireboltClient() as fc:
    debug(fc.settings)

### Run a query

In [ ]:
from firebolt.firebolt_client import FireboltClient
from firebolt.model.engine import Engine

sql = "SELECT 1;"

with FireboltClient():
    engine = Engine.get_by_name(engine_name=engine_name)
    response = engine.run_query(sql=sql)

debug(response)

### Get existing Engine by name, start, stop

In [ ]:
from firebolt.firebolt_client import FireboltClient
from firebolt.model.engine import Engine

with FireboltClient():
    engine = Engine.get_by_name(engine_name=engine_name)
    debug(engine)

    started_engine = engine.start()
    debug(started_engine)

    stopped_engine = engine.stop()
    debug(stopped_engine)

### Inspect engine revision

In [ ]:
with FireboltClient():
    debug(engine.get_latest_engine_revision().compute_instance_type_name)

### Create default general purpose (ingest) engine
Note: there is also `Engine.create_analytics(...)` which works the same way as `create_general_purpose(...)`.

In [ ]:
from firebolt.firebolt_client import FireboltClient
from firebolt.model.engine import Engine

with FireboltClient():
    engine = Engine.create_general_purpose(
        name=engine_name, compute_instance_type_name="m5d.4xlarge"
    )
    debug(engine)

### Copy an Engine

In [ ]:
from firebolt.firebolt_client import FireboltClient
from firebolt.model.engine import Engine

with FireboltClient():
    engine = Engine.get_by_name(engine_name=engine_name)
    engine.name = f"{engine_name}_copy"
    debug(engine)
    debug(engine.create_with_latest_revision())

### Get engines for a database

In [ ]:
from firebolt.firebolt_client import FireboltClient
from firebolt.model.database import Database

with FireboltClient():
    database = Database.get_by_name(database_name=database_name)
    engines = database.engines

debug(engines)

### Create engine, database, and bind them

In [ ]:
from firebolt.firebolt_client import FireboltClient
from firebolt.model.database import Database
from firebolt.model.engine import Engine

with FireboltClient():
    database = Database.get_by_name(database_name=database_name)
    engine = Engine.create_general_purpose(
        name=engine_name, compute_instance_type_name="m5d.4xlarge"
    )
    binding = engine.bind_to_database(database=database, is_default_engine=True)
    debug(binding)

### Get default provider, region, and an instance_type
An example of these might be:
 * provider: AWS
 * region: us-east-1
 * instance_type: i3.4xlarge

In [ ]:
from firebolt.firebolt_client import FireboltClient
from firebolt.model.provider import providers
from firebolt.model.region import regions
from firebolt.model.instance_type import instance_types

with FireboltClient():
    debug(providers.default_provider)
    debug(regions.default_region)
    debug(instance_types.get_by_name(instance_type_name="i3.4xlarge"))